# Solving A Simple Version of the Gauntlet

This notebook describes a mathematical and programatic solution to the gauntlet. 

# Mathematical Description of Action Optimizer

The `ActionOptimizer` algorithm is designed to find an optimal sequence of actions in a discrete-time, state-based scenario, commonly found in game-like environments. The approach combines recursive search with memoization and iterative deepening.

## Problem Setup

- Let $X(n)$ represent the state vector at discrete time step $n$.
- $N$ is the terminal time step, which is not fixed but determined dynamically based on the state $X$.
- At each time step, the system can perform one of $M$ actions, transforming $X(n)$ to $X(n+1)$.
- The objective function, $O(X(N))$, calculates the score of a state at terminal time $N$, and the goal is to maximize this score.

## Recursive Algorithm with Memoization

### State Transformation
- State transformation is governed by the function $evolve(X, action)$: 
  - $X_{new} = evolve(X, action)$
- This function is part of the `Game` class, encapsulating the rules of state evolution.

### Terminal State Check
- The terminal state check is performed by $is\_sequence\_ended(X)$:
  - Returns `True` if $X$ is a terminal state; otherwise, `False`.

### Scoring Function
- The scoring function $O(X)$ computes the score for a state $X$:
  - $score = O(X)$
- This function is also encapsulated within the `Game` class.

### Recursive Function
- The core of the algorithm is the recursive function $explore(X, current\_sequence)$:
  - It explores all possible action sequences starting from a given state $X$.
  - Utilizes memoization to store and retrieve results of previously explored states.
  - Terminates when $is\_sequence\_ended(X)$ is `True` and evaluates the score.

### Memoization
- A memoization dictionary, $memo$, caches the scores of explored states to reduce computational redundancy.
- Before exploring a state, the algorithm checks $memo$ to avoid re-exploration.

## Iterative Deepening Technique

- Iterative deepening is implemented via $explore\_iterative(max\_depth)$:
  - The algorithm incrementally deepens the exploration depth from 1 to $max\_depth$.
  - This ensures that shallower optimal solutions are found first, managing the depth and complexity of recursion.

## Integration with the Game Class

- The `Game` class abstracts the specifics of state evolution, action possibilities, terminal condition, and scoring.
- Placeholders in the `Game` class:
  - $evolve(X, action)$: Defines how the state $X$ changes with an action.
  - $get\_actions(X)$: Lists possible actions for state $X$.
  - $is\_sequence\_ended(X)$: Determines if $X$ is a terminal state.
  - $compute\_score(X)$: Calculates the score of state $X$.

## Execution

- Initialize `Game` with the initial state $X(0)$.
- Instantiate `ActionOptimizer` with the `Game` object.
- Invoke `find_optimal_sequences` to compute the optimal sequence of actions, choosing between recursive and iterative deepening modes.



In [2]:
from typing import List, Tuple, Any

GameState = Tuple[Any, ...]  # Type alias for the game state
ActionOutcome = Tuple[GameState, float]  # Type alias for action probability outcomes
ActionSequence = List[int]  # Type alias for a sequence of actions

In [3]:
class ActionOptimizer:
    def __init__(self, game):
        self.game = game
        self.optimal_sequences = []
        self.max_score = float('-inf')
        self.memo = {}

    def explore_recursive(self):
        def explore(X, current_sequence):
            X_key = tuple(X)
            if X_key in self.memo:
                return self.memo[X_key]

            if self.game.is_sequence_ended(X):
                score = self.game.compute_score(X)
                self.update_optimal_sequences(score, current_sequence)
                self.memo[X_key] = score
                return score

            for action in self.game.get_actions(X):
                new_X = self.game.evolve(X, action)
                current_sequence.append(action)
                explore(new_X, current_sequence)
                current_sequence.pop()

            self.memo[X_key] = None

        explore(self.game.initial_state, [])

    def explore_iterative(self, max_depth):
        def explore_depth_limited(X, current_sequence, depth):
            if depth == 0:
                return

            X_key = tuple(X)
            if X_key in self.memo:
                return self.memo[X_key]

            if self.game.is_sequence_ended(X):
                score = self.game.compute_score(X)
                self.update_optimal_sequences(score, current_sequence)
                self.memo[X_key] = score
                return score

            for action in self.game.get_actions(X):
                new_X = self.game.evolve(X, action)
                current_sequence.append(action)
                explore_depth_limited(new_X, current_sequence, depth - 1)
                current_sequence.pop()

            self.memo[X_key] = None

        for depth in range(1, max_depth + 1):
            explore_depth_limited(self.game.initial_state, [], depth)

    def update_optimal_sequences(self, score, current_sequence):
        if score > self.max_score:
            self.max_score = score
            self.optimal_sequences = [current_sequence.copy()]
        elif score == self.max_score:
            self.optimal_sequences.append(current_sequence.copy())

    def find_optimal_sequences(self, iterative_deepening=False, max_depth=100):
        if iterative_deepening:
            self.explore_iterative(max_depth)
        else:
            self.explore_recursive()
        return self.optimal_sequences, self.max_score


In [4]:
ActionSequence = List[int]  # Type alias for a sequence of actions

class ProbabilisticActionOptimizer:
    def __init__(self, game: Game):
        self.game = game
        self.optimal_sequences: List[ActionSequence] = []
        self.max_expected_score: float = float('-inf')

    def expected_score(self, sequence: ActionSequence, probability: float) -> float:
        assert self.game.is_sequence_ended(sequence[-1]), \
            "Expected score should be calculated at the end of the game sequence."
        
        final_state_score = self.game.compute_score(sequence[-1])
        weighted_score = final_state_score * probability
        return weighted_score

    def explore(self, X: GameState, current_sequence: ActionSequence, probability: float = 1.0):
        if self.game.is_sequence_ended(X):
            exp_score = self.expected_score(current_sequence, probability)
            if exp_score > self.max_expected_score:
                self.max_expected_score = exp_score
                self.optimal_sequences = [current_sequence.copy()]
            elif exp_score == self.max_expected_score:
                self.optimal_sequences.append(current_sequence.copy())
            return

        for action in self.game.get_actions(X):
            for new_X, action_probability in self.game.probabilistic_evolve(X, action):
                current_sequence.append(action)
                new_probability = probability * action_probability
                self.explore(new_X, current_sequence, new_probability)
                current_sequence.pop()

    def find_optimal_sequences(self) -> Tuple[List[ActionSequence], float]:
        self.explore(self.game.initial_X, [])
        return self.optimal_sequences, self.max_expected_score


NameError: name 'Game' is not defined

# Specific Game Implementation

In [5]:
class GameState:
    def __init__(self, initial_state: Any):
        self.state = initial_state

    def move(self, action: int) -> 'GameState':
        # Implement the logic to modify the game state based on the action
        # Return a new GameState object representing the new state
        # Example implementation (modify as needed):
        new_state = ... # Logic to determine the new state
        return GameState(new_state)

    # Add additional methods as needed for state management


In [6]:
class Game:
    def __init__(self, initial_state: Any):
        self.initial_X = GameState(initial_state)

    def get_actions(self, X: GameState) -> List[int]:
        # Return possible actions based on state X
        return [0, 1, 2]  # Example actions

    def probabilistic_evolve(self, X: GameState, action: int) -> List[Tuple[GameState, float]]:
        # Given a state and an action, return a list of tuples (new_X, probability)
        # for each possible outcome
        return [(X.move(action), 0.5), (X.move(action), 0.5)]  # Example outcomes

    def is_sequence_ended(self, X: GameState) -> bool:
        # Determine if the current state signifies the end of a sequence
        return False  # Example condition

    def compute_score(self, state: GameState) -> float:
        # Calculate and return the score for the given game state
        return 0.0  # Example score calculation

In [7]:


# Example Usage
game = Game(initial_state)  # Initialize the Game with the initial state
optimizer = ActionOptimizer(game)

# Using Recursive Approach
optimal_sequences, optimal_score = optimizer.find_optimal_sequences()
print("Recursive Approach - Optimal sequences:", optimal_sequences)
print("Optimal score:", optimal_score)

# Using Iterative Deepening Approach
optimal_sequences, optimal_score = optimizer.find_optimal_sequences(iterative_deepening=True, max_depth=100)
print("Iterative Deepening - Optimal sequences:", optimal_sequences)
print("Optimal score:", optimal_score)


NameError: name 'initial_state' is not defined

In [8]:
initial_state = ...  # Define the initial state appropriate for your game

game = Game(initial_state)
optimizer = ProbabilisticActionOptimizer(game)

optimal_sequences, optimal_score = optimizer.find_optimal_sequences()

print("Optimal sequences:", optimal_sequences)
print("Optimal expected score:", optimal_score)

NameError: name 'ProbabilisticActionOptimizer' is not defined